# Redes Neurais Artificiais 2020.1 -- Projeto Prático 3.4

**Disciplina**: Redes Neurais Artificiais 2020.1  
**Professora**: Elloá B. Guedes (ebgcosta@uea.edu.br)  
**Github**: http://github.com/elloa  
        

Levando em conta a base de dados **_Forest Cover Type_**, esta terceira parte do Projeto Prático 3 diz respeito à proposição e avaliação de múltiplas redes neurais artificiais do tipo feedforward multilayer perceptron para o problema da classificação multi-classe da cobertura florestal em uma área do Roosevelt National Forest.

## Busca em Grade

Uma maneira padrão de escolher os parâmetros de um modelo de Machine Learning é por meio de uma busca em grade via força bruta. O algoritmo da busca em grade é dado como segue:

1. Escolha a métrica de desempenho que você deseja maximizar  
2. Escolha o algoritmo de Machine Learning (exemplo: redes neurais artificiais). Em seguida, defina os parâmetros ou hiperparâmetros deste tipo de modelo sobre os quais você deseja otimizar (número de épocas, taxa de aprendizado, etc.) e construa um array de valores a serem testados para cada parâmetro ou hiperparâmetro.  
3. Defina a grade de busca, a qual é dada como o produto cartesiano de cada parâmetro a ser testado. Por exemplo, para os arrays [50, 100, 1000] e [10, 15], tem-se que a grade é [(50,10), (50,15), (100,10), (100,15), (1000,10), (1000,15)].
4. Para cada combinação de parâmetros a serem otimizados, utilize o conjunto de treinamento para realizar uma validação cruzada (holdout ou k-fold) e calcule a métrica de avaliação no conjunto de teste (ou conjuntos de teste)
5. Escolha a combinação de parâmetros que maximizam a métrica de avaliação. Este é o modelo otimizado.

Por que esta abordagem funciona? Porque a busca em grade efetua uma pesquisa extensiva sobre as possíveis combinações de valores para cada um dos parâmetros a serem ajustados. Para cada combinação, ela estima a performance do modelo em dados novos. Por fim, o modelo com melhor métrica de desempenho é escolhido. Tem-se então que este modelo é o que melhor pode vir a generalizar mediante dados nunca antes vistos.

## Efetuando a Busca em Grade sobre Hiperparâmetros das Top-6 RNAs

Considerando a etapa anterior do projeto prático, foram identificadas pelo menos 6 melhores Redes Neurais para o problema da classificação multi-classe da cobertura florestal no conjunto de dados selecionado. Algumas destas redes possuem atributos categóricos como variáveis preditoras, enquanto outras possuem apenas os atributos numéricos como preditores.

A primeira etapa desta segunda parte do projeto consiste em trazer para este notebook estas seis arquiteturas, ressaltando:

1. Número de neurônios ocultos por camada  
2. Função de Ativação  
3. Utilização ou não de atributos categóricos   
4. Desempenho médio +- desvio padrão nos testes anteriores  
5. Número de repetições que a equipe conseguiu realizar para verificar os resultados  

Elabore uma busca em grade sobre estas arquiteturas que contemple variações nos hiperparâmetros a seguir, conforme documentação de [MLPClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)

A. Solver  (Não usar o LBFGS, pois é mais adequado para datasets pequenos)  
B. Batch Size  
C. Learning Rate Init  
D. Paciência (n_iter_no_change)  
E. Épocas  

Nesta busca em grande, contemple a utilização do objeto [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)

## Bibliotecas

In [40]:
import warnings
warnings.filterwarnings('ignore')

import re

import numpy as np
import pandas as pd

from itertools import product

from joblib import dump, load

from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

from sklearn.metrics import f1_score

## Base de Dados

In [2]:
database = pd.read_csv('../database/covtype.csv')
database

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Soil_Type1,Soil_Type2,Soil_Type3,Soil_Type4,Soil_Type5,Soil_Type6,Soil_Type7,Soil_Type8,Soil_Type9,Soil_Type10,Soil_Type11,Soil_Type12,Soil_Type13,Soil_Type14,Soil_Type15,Soil_Type16,Soil_Type17,Soil_Type18,Soil_Type19,Soil_Type20,Soil_Type21,Soil_Type22,Soil_Type23,Soil_Type24,Soil_Type25,Soil_Type26,Soil_Type27,Soil_Type28,Soil_Type29,Soil_Type30,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2396,153,20,85,17,108,240,237,118,837,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
581008,2391,152,19,67,12,95,240,237,119,845,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
581009,2386,159,17,60,7,90,236,241,130,854,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
581010,2384,170,15,60,5,90,230,245,143,864,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3


## Busca em Grade

- Escolha a métrica de desempenho que você deseja maximizar  

In [3]:
# Definindo a métrica como F1-Score
metrica = 'f1'

- Escolha o algoritmo de Machine Learning (exemplo: redes neurais artificiais). Em seguida, defina os parâmetros ou hiperparâmetros deste tipo de modelo sobre os quais você deseja otimizar (número de épocas, taxa de aprendizado, etc.) e construa um array de valores a serem testados para cada parâmetro ou hiperparâmetro.  

In [4]:
# Definindo o algoritmo de Machine Learning
algoritmo = MLPClassifier()

# Definindo os parâmetros
parametros = {'hidden_layer_sizes': [(16, 6)],    # Camadas
              'activation': ['tanh'],             # Funções de ativação
              'solver': ['sgd'],                  # Otimizadores
              'max_iter': [250, 500],             # Épocas
              'shuffle': [True],                  # Condição para embaralhar as amostras em cada iteração
              'n_iter_no_change': [5, 10]}        # Máximo de épocas sem melhorias na rede
            

- Defina a grade de busca, a qual é dada como o produto cartesiano de cada parâmetro a ser testado. Por exemplo, para os arrays [50, 100, 1000] e [10, 15], tem-se que a grade é [(50,10), (50,15), (100,10), (100,15), (1000,10), (1000,15)].

In [5]:
# Método que retorna o produto cartesiano dos parâmetros
def produto(parametros):
    params = list (dict(zip(parametros.keys(), values)) for values in product(*parametros.values()))
    for param in params:
        for key, value in param.items():
            param[key] = [value]
    return params
        
# Definindo a grade de busca
grade_busca = produto(parametros)

In [6]:
grade_busca

[{'activation': ['tanh'],
  'hidden_layer_sizes': [(16, 6)],
  'max_iter': [250],
  'n_iter_no_change': [5],
  'shuffle': [True],
  'solver': ['sgd']},
 {'activation': ['tanh'],
  'hidden_layer_sizes': [(16, 6)],
  'max_iter': [250],
  'n_iter_no_change': [10],
  'shuffle': [True],
  'solver': ['sgd']},
 {'activation': ['tanh'],
  'hidden_layer_sizes': [(16, 6)],
  'max_iter': [500],
  'n_iter_no_change': [5],
  'shuffle': [True],
  'solver': ['sgd']},
 {'activation': ['tanh'],
  'hidden_layer_sizes': [(16, 6)],
  'max_iter': [500],
  'n_iter_no_change': [10],
  'shuffle': [True],
  'solver': ['sgd']}]

- Para cada combinação de parâmetros a serem otimizados, utilize o conjunto de treinamento para realizar uma validação cruzada (holdout ou k-fold) e calcule a métrica de avaliação no conjunto de teste (ou conjuntos de teste)

In [7]:
# Alocando os atributos preditores na variável X
X = database.iloc[:1000,:-1]

# Alocando o atributo alvo na variável y
y = database.iloc[:1000,-1:]

# Definindo a busca em grade com os parâmetros e a métrica
search = GridSearchCV(algoritmo, grade_busca, scoring='%s_macro' % metrica)

# Definindo a partição holdout 70/30
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, shuffle=True)
    
# Buscando a melhor configuração de parâmetros
search.fit(X_train, y_train)

# Calculando a métrica de avaliação
y_pred = search.predict(X_test)
f1_score(y_test, y_pred, average='macro')

0.24293785310734464

- Escolha a combinação de parâmetros que maximizam a métrica de avaliação. Este é o modelo otimizado.

In [8]:
# Parâmetros que otimizam o desempenho do modelo Multilayer Perceptron
search.best_params_

{'activation': 'tanh',
 'hidden_layer_sizes': (16, 6),
 'max_iter': 500,
 'n_iter_no_change': 10,
 'shuffle': True,
 'solver': 'sgd'}

## Efetuando a Busca em Grade sobre Hiperparâmetros das Top-6 RNAs

Considerando a etapa anterior do projeto prático, foram identificadas pelo menos 6 melhores Redes Neurais para o problema da classificação multi-classe da cobertura florestal no conjunto de dados selecionado. Algumas destas redes possuem atributos categóricos como variáveis preditoras, enquanto outras possuem apenas os atributos numéricos como preditores.

A primeira etapa desta segunda parte do projeto consiste em trazer para este notebook estas seis arquiteturas, ressaltando:

In [9]:
# Leitura das arquiteturas
arquiteturas = [load('../redes/rede{0}.joblib'.format(i)) for i in range(1, 7)]

- Número de neurônios ocultos por camada  

In [10]:
[arquiteturas[i].hidden_layer_sizes for i in range(0,6)]

[(9, 10), (13,), (9, 10), (25,), (25,), (19, 6)]

- Função de Ativação  

In [11]:
[arquiteturas[i].activation for i in range(0,6)]

['tanh', 'tanh', 'tanh', 'logistic', 'logistic', 'tanh']

- Utilização ou não de atributos categóricos   

    Redes treinadas com atributos categóricos

In [12]:
for i in range(3):
    print('Rede {0}'.format(i+1), arquiteturas[i])

Rede 1 MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(9, 10), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=3, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)
Rede 2 MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(13,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=150,
              momentum=0.9, n_iter_no_change=3, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validati

    Redes treinadas sem atributos categóricos

In [13]:
for i in range(3):
    print('Rede {0}'.format(i+4), arquiteturas[i])

Rede 4 MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(9, 10), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=3, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)
Rede 5 MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(13,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=150,
              momentum=0.9, n_iter_no_change=3, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validati

- Desempenho médio +- desvio padrão nos testes anteriores  
  

- Número de repetições que a equipe conseguiu realizar para verificar os resultados

    Definiu-se 10 repetições para verificar os resultados.

Elabore uma busca em grade sobre estas arquiteturas que contemple variações nos hiperparâmetros a seguir, conforme documentação de [MLPClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)

A. Solver  (Não usar o LBFGS, pois é mais adequado para datasets pequenos)  
B. Batch Size  
C. Learning Rate Init  
D. Paciência (n_iter_no_change)  
E. Épocas  

In [14]:
# Definindo os hiperparâmetros
hiperparametros = { 'solver': ['sgd', 'adam'],            
                    'batch_size': [500],                      
                    'learning_rate': ['constant', 'invscaling', 'adaptive'],                    
                    'n_iter_no_change': [5],
                    'max_iter': [100]} 

In [15]:
indexs = []
for i in range(1,8):
    indexs.append(database[database.Cover_Type == i][:1000])
database_min = pd.concat(indexs)

- Busca em Grade com os atributos categóricos

In [16]:
# Definindo a métrica como F1-Score
metrica = 'f1'

# Combinações dos hiperparâmetros
grades = produto(hiperparametros)

# Melhores hiperparâmetros para cada arquitetura
bests1 = []

# Alocando os atributos preditores na variável X
X = database_min.iloc[:,:-1]

# Alocando o atributo alvo na variável y
y = database_min.iloc[:,-1:]

for i in range(3,6):
    print(i)
    # Definindo a busca em grade para as arquitetura 4, 5 e 6
    search = GridSearchCV(arquiteturas[i], grades, scoring='%s_macro' % metrica)
    
    # Definindo a partição holdout 70/30
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
    
    # Buscando a melhor configuração de hiperparâmetros
    search.fit(X_train, y_train)

    # Salvando os melhores hiperparâmetros de cada arquitetura
    for key, value in search.best_params_.items():
        search.best_params_[key] = [value]
    bests1.append(search.best_params_)

3
4
5


- Melhores hiperparâmetros com os atributos categóricos

In [17]:
for i in range(3,6):
    print('Rede {0}'.format(i+1), bests1[i-3])

Rede 4 {'batch_size': [500], 'learning_rate': ['constant'], 'max_iter': [100], 'n_iter_no_change': [5], 'solver': ['adam']}
Rede 5 {'batch_size': [500], 'learning_rate': ['constant'], 'max_iter': [100], 'n_iter_no_change': [5], 'solver': ['adam']}
Rede 6 {'batch_size': [500], 'learning_rate': ['invscaling'], 'max_iter': [100], 'n_iter_no_change': [5], 'solver': ['adam']}


- Busca em Grade sem os atributos categóricos

In [18]:
database2 = database_min.copy()

# Removendo os atributos categóricos
columns_to_drop = []
for column in list(database2.columns):
    column_search = re.search('Soil_Type|Wilderness_Area', column)
    if column_search:
        columns_to_drop.append(column)
database2.drop(columns=columns_to_drop, axis=1, inplace=True)

In [19]:
# Definindo a métrica como F1-Score
metrica = 'f1'

# Combinações dos hiperparâmetros
grades = produto(hiperparametros)

# Melhores hiperparâmetros para cada arquitetura
bests2 = []

# Alocando os atributos preditores na variável X
X = database2.iloc[:,:-1]

# Alocando o atributo alvo na variável y
y = database2.iloc[:,-1:]

for i in range(0,3):
    print(i)

    # Definindo a busca em grade para as arquitetura 1, 2 e 3
    search = GridSearchCV(arquiteturas[i], grades, scoring='%s_macro' % metrica)
    
    # Definindo a partição holdout 70/30
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
    
    # Buscando a melhor configuração de hiperparâmetros
    search.fit(X_train, y_train)

     # Salvando os melhores hiperparâmetros de cada arquitetura
    for key, value in search.best_params_.items():
        search.best_params_[key] = [value]
    bests2.append(search.best_params_)

0
1
2


- Melhores hiperparâmetros sem os atributo categóricos

In [20]:
for i in range(0,3):
    print('Rede {0}'.format(i+1), bests2[i-3])

Rede 1 {'batch_size': [500], 'learning_rate': ['constant'], 'max_iter': [100], 'n_iter_no_change': [5], 'solver': ['adam']}
Rede 2 {'batch_size': [500], 'learning_rate': ['constant'], 'max_iter': [100], 'n_iter_no_change': [5], 'solver': ['adam']}
Rede 3 {'batch_size': [500], 'learning_rate': ['adaptive'], 'max_iter': [100], 'n_iter_no_change': [5], 'solver': ['adam']}


## Validação Cruzada k-fold

Na elaboração da busca em grid, vamos avaliar os modelos propostos segundo uma estratégia de validação cruzada ainda não explorada até o momento: a validação cruzada k-fold. Segundo a mesma, o conjunto de dados é particionado em k partes: a cada iteração, separa-se uma das partes para teste e o modelo é treinado com as k-1 partes remanescentes. Valores sugestivos de k na literatura são k = 3, 5 ou 10, pois o custo computacional desta validação dos modelos é alto. A métrica de desempenho é resultante da média dos desempenhos nas k iterações. A figura a seguir ilustra a ideia desta avaliação

<img src = "https://ethen8181.github.io/machine-learning/model_selection/img/kfolds.png" width=600></img>

Considerando a métrica de desempenho F1-Score, considere a validação cruzada 5-fold para aferir os resultados da busca em grande anterior.

- Validação cruzada com atributos categóricos

In [21]:
metrica = 'f1'

# Desempenho de cada arquitetura
fscore = dict()

for i in range(1,7):
    fscore.setdefault('rede{0}'.format(i), [])

In [22]:
# Definindo o número de folds
k_folds = KFold(5)

# Alocando os atributos preditores na variável X
X = database_min.iloc[:,:-1]

# Alocando o atributo alvo na variável y
y = database_min.iloc[:,-1:]

rede = 4

# Validação Cruzada com os atributos categóricos
for params, arquitetura in zip(bests1, arquiteturas[3:6]):
    print(params)
    print('rede ', rede)
    
    search = GridSearchCV(arquitetura, params, scoring='%s_macro' % metrica)
    
    fscores = []
    
    for k, (train, test) in enumerate(k_folds.split(X, y)):
        
        print('k = {0}'.format(k+1))

        X_train = X.iloc[train]
        y_train = y.iloc[train]
        
        
        search.fit(X_train, y_train)
        
        X_test = X.iloc[test]
        y_test = y.iloc[test]


        y_pred = search.predict(X_test)

        fscores.append(f1_score(y_pred, y_test, average='macro'))

    fscore['rede'+str(rede)].append((fscores, search.best_params_, arquitetura))
    rede += 1

{'batch_size': [500], 'learning_rate': ['constant'], 'max_iter': [100], 'n_iter_no_change': [5], 'solver': ['adam']}
rede  4
k = 1
k = 2
k = 3
k = 4
k = 5
{'batch_size': [500], 'learning_rate': ['constant'], 'max_iter': [100], 'n_iter_no_change': [5], 'solver': ['adam']}
rede  5
k = 1
k = 2
k = 3
k = 4
k = 5
{'batch_size': [500], 'learning_rate': ['invscaling'], 'max_iter': [100], 'n_iter_no_change': [5], 'solver': ['adam']}
rede  6
k = 1
k = 2
k = 3
k = 4
k = 5


- Validação cruzada sem atributos categóricos

In [23]:
# Definindo o número de folds
k_folds = KFold(5)

# Alocando os atributos preditores na variável X
X = database2.iloc[:,:-1]

# Alocando o atributo alvo na variável y
y = database2.iloc[:,-1:]

rede = 1

# Validação Cruzada com os atributos categóricos
for params, arquitetura in zip(bests2, arquiteturas[0:3]):
    
    print('rede ', rede)
    
    search = GridSearchCV(arquitetura, params, scoring='%s_macro' % metrica)
    
    fscores = []
    
    for k, (train, test) in enumerate(k_folds.split(X, y)):
        
        print('k = {0}'.format(k+1))

        X_train = X.iloc[train]
        y_train = y.iloc[train]
        
        
        search.fit(X_train, y_train)
        
        X_test = X.iloc[test]
        y_test = y.iloc[test]


        y_pred = search.predict(X_test)

        fscores.append(f1_score(y_pred, y_test, average='macro'))

    fscore['rede'+str(rede)].append((fscores, search.best_params_, arquitetura))
    rede += 1

rede  1
k = 1
k = 2
k = 3
k = 4
k = 5
rede  2
k = 1
k = 2
k = 3
k = 4
k = 5
rede  3
k = 1
k = 2
k = 3
k = 4
k = 5


- Resultados

In [24]:
fscore

{'rede1': [([0.0, 0.0009492168960607499, 0.007042253521126762, 0.0, 0.0],
   {'batch_size': 500,
    'learning_rate': 'constant',
    'max_iter': 100,
    'n_iter_no_change': 5,
    'solver': 'adam'},
   MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
                 beta_2=0.999, early_stopping=False, epsilon=1e-08,
                 hidden_layer_sizes=(9, 10), learning_rate='constant',
                 learning_rate_init=0.001, max_fun=15000, max_iter=200,
                 momentum=0.9, n_iter_no_change=3, nesterovs_momentum=True,
                 power_t=0.5, random_state=None, shuffle=True, solver='adam',
                 tol=0.0001, validation_fraction=0.1, verbose=False,
                 warm_start=False))],
 'rede2': [([0.04608294930875576, 0.0, 0.03592265498828767, 0.0, 0.0],
   {'batch_size': 500,
    'learning_rate': 'constant',
    'max_iter': 100,
    'n_iter_no_change': 5,
    'solver': 'adam'},
   MLPClassifier(activation='tanh', alpha=0.0001

## Identificando a mellhor solução

Como resultado da busca em grande com validação cruzada 5-fold, identifique o modelo otimizado com melhor desempenho para o problema. Apresente claramente este modelo, seus parâmetros, hiperparâmetros otimizados e resultados para cada um dos folds avaliados. Esta é a melhor solução identificada em decorrência deste projeto

In [56]:
desempenho = dict()
for key, value in fscore.items():
    desempenho.setdefault(key, np.mean(value[0][0]))
best_rede = sorted(desempenho.items(), key=lambda x: x[1], reverse=True)[0]
best_rede

('rede4', 0.02315843978735526)

- Modelo, seus parâmetros, hiperparâmetros otimizados

In [64]:
fscore[best_rede[0]][0][2]

MLPClassifier(activation='logistic', alpha=0.0001, batch_size='auto',
              beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(25,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=2, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [63]:
fscore[best_rede[0]][0][1]

{'batch_size': 500,
 'learning_rate': 'constant',
 'max_iter': 100,
 'n_iter_no_change': 5,
 'solver': 'adam'}

- Resultados para cada um dos folds avaliados

In [65]:
fscore[best_rede[0]][0][0]

[0.04305200341005968,
 0.0024783147459727386,
 0.0473293185157592,
 0.003273322422258592,
 0.019659239842726082]

## Empacotando a solução

Suponha que você deve entregar este classificador ao órgão responsável por administrar o Roosevelt National Park. Para tanto, você deve fazer uma preparação do mesmo para utilização neste cenário. Uma vez que já identificou os melhores parâmetros e hiperparâmetros, o passo remanescente consiste em treinar o modelo com estes valores e todos os dados disponíveis, salvando o conjunto de pesos do modelo ao final para entrega ao cliente. Assim, finalize o projeto prático realizando tais passos.

1. Consulte a documentação a seguir:
https://scikit-learn.org/stable/modules/model_persistence.html
2. Treine o modelo com todos os dados
3. Salve o modelo em disco  
4. Construa uma rotina que recupere o modelo em disco  
5. Mostre que a rotina é funcional, fazendo previsões com todos os elementos do dataset e exibindo uma matriz de confusão das mesmas

- Treine o modelo com todos os dados

In [ ]:
# Alocando os atributos preditores na variável X
X = database.iloc[:,:-1]

# Alocando o atributo alvo na variável y
y = database.iloc[:,-1:]

# Definindo a partição holdout 70/30
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

# Treinando o modelo com melhor desempenho
melhor_modelo.fit(X_train, X_test)


- Salve o modelo em disco

In [ ]:
# Salvando o modelo no diretório redes
dump(melhor_modelo, '../redes/melhor_modelo.joblib') 

- Construa uma rotina que recupere o modelo em disco 

In [ ]:
# Recuperando o modelo salvo em disco
def get_model():
    return load('../redes/melhor_modelo.joblib')

- Mostre que a rotina é funcional, fazendo previsões com todos os elementos do dataset e exibindo uma matriz de confusão das mesmas


In [ ]:
# Carregando o modelo
model = get_model()

# Alocando os atributos preditores na variável X
X = database.iloc[:,:-1]

# Alocando o atributo alvo na variável y
y = database.iloc[:,-1:]

# Definindo a partição holdout 70/30
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

# Treinando o modelo
model.fit(X_train, y_train)

# Construindo a matrix de confusão
fig, ax = plt.subplots(figsize=(7,7))
plot_confusion_matrix(model, X_test, y_test, ax=ax, cmap='YlGnBu', values_format="")
plt.title('Matriz de Confusão')
plt.show()